In [81]:
INPUT_TRAIN = "./input/train_v2.csv"
INPUT_TEST = "./input/test_v2.csv"

TRAIN='train-processed.csv'
TEST='test-processed.csv'
Y='target.csv'

In [1]:
import os
import gc
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

import warnings
warnings.filterwarnings('ignore')

In [18]:
def qv(df, n_of_rows=5):   #quick_review
    display(df.head(n_of_rows))
    print(df.shape)

In [61]:
def load_df(csv_path='./input/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]

        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

# print(os.listdir("./input"))
                     

In [92]:
def load_and_check_df(csv_path='./input/train_v2.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        print ([x for x in column_as_df.columns])
#         column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]

#         df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [93]:
train_df = load_and_check_df(nrows=2)
qv(train_df)

['browser', 'browserSize', 'browserVersion', 'deviceCategory', 'flashVersion', 'isMobile', 'language', 'mobileDeviceBranding', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'mobileDeviceModel', 'mobileInputSelector', 'operatingSystem', 'operatingSystemVersion', 'screenColors', 'screenResolution']
['city', 'cityId', 'continent', 'country', 'latitude', 'longitude', 'metro', 'networkDomain', 'networkLocation', 'region', 'subContinent']
['bounces', 'hits', 'newVisits', 'pageviews', 'sessionQualityDim', 'timeOnSite', 'visits']
['adwordsClickInfo.criteriaParameters', 'campaign', 'keyword', 'medium', 'referralPath', 'source']
Loaded train_v2.csv. Shape: (2, 13)


,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{'browser': 'Firefox', 'browserVersion': 'not ...",3162355547410993243,"{'continent': 'Europe', 'subContinent': 'Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","{'campaign': '(not set)', 'source': 'google', ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{'browser': 'Chrome', 'browserVersion': 'not a...",8934116514970143966,"{'continent': 'Americas', 'subContinent': 'Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{'visits': '1', 'hits': '2', 'pageviews': '2',...",{'referralPath': '/a/google.com/transportation...,1508176307,6,1508176307


(2, 13)


In [88]:
# This function is just a packaged version of this kernel:
# https://www.kaggle.com/fabiendaniel/lgbm-rf-starter-lb-1-70
def process_dfs(train_df, test_df):
    print("Processing dfs...")
    print("Dropping repeated columns...")
    columns = [col for col in train_df.columns if train_df[col].nunique() > 1]
    
    train_df = train_df[columns]
    test_df = test_df[columns]

    trn_len = train_df.shape[0]
    merged_df = pd.concat([train_df, test_df])

    merged_df['diff_visitId_time'] = merged_df['visitId'] - merged_df['visitStartTime']
    merged_df['diff_visitId_time'] = (merged_df['diff_visitId_time'] != 0).astype(int)
    del merged_df['visitId']

    del merged_df['sessionId']

    print("Generating date columns...")
    format_str = '%Y%m%d' 
    merged_df['formated_date'] = merged_df['date'].apply(lambda x: datetime.strptime(str(x), format_str))
    merged_df['WoY'] = merged_df['formated_date'].apply(lambda x: x.isocalendar()[1])
    merged_df['month'] = merged_df['formated_date'].apply(lambda x:x.month)
    merged_df['quarter_month'] = merged_df['formated_date'].apply(lambda x:x.day//8)
    merged_df['weekday'] = merged_df['formated_date'].apply(lambda x:x.weekday())

    del merged_df['date']
    del merged_df['formated_date']

    merged_df['formated_visitStartTime'] = merged_df['visitStartTime'].apply(
        lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)))
    merged_df['formated_visitStartTime'] = pd.to_datetime(merged_df['formated_visitStartTime'])
    merged_df['visit_hour'] = merged_df['formated_visitStartTime'].apply(lambda x: x.hour)

    del merged_df['visitStartTime']
    del merged_df['formated_visitStartTime']

    print("Encoding columns with pd.factorize()")
    for col in merged_df.columns:
        if col in ['fullVisitorId', 'month', 'quarter_month', 'weekday', 'visit_hour', 'WoY']: continue
        if merged_df[col].dtypes == object or merged_df[col].dtypes == bool:
            merged_df[col], indexer = pd.factorize(merged_df[col])

    print("Splitting back...")
    train_df = merged_df[:trn_len]
    test_df = merged_df[trn_len:]
    return train_df, test_df

def preprocess(check_n_rows):
    train_df = load_df(nrows=check_n_rows)
    test_df = load_df(INPUT_TEST, nrows=check_n_rows)

    target = train_df['totals.transactionRevenue'].fillna(0).astype(float)
    target = target.apply(lambda x: np.log1p(x))
    del train_df['totals.transactionRevenue']

#     train_df, test_df = process_dfs(train_df, test_df)
#     train_df.to_csv(TRAIN, index=False)
#     test_df.to_csv(TEST, index=False)
#     target.to_csv(Y, index=False)

In [89]:
preprocess(5)

Loaded train_v2.csv. Shape: (5, 50)
Loaded test_v2.csv. Shape: (5, 50)


KeyError: 'totals.transactionRevenue'

In [80]:
a = list(load_df(nrows=3))
b = list(load_df(nrows=2))

# print (a)

list_a = [1,2,3]
list_b = [1,2,4]


set(a).symmetric_difference(b)

Loaded train_v2.csv. Shape: (3, 50)
Loaded train_v2.csv. Shape: (2, 49)


{'trafficSource.isTrueDirect'}